#Задача 1.

Изследвайте системата за съвместимост в зависимост от параметъра и я решете:

$$
\begin{cases}
2 x_1+5 x_2-8 x_3=8 \\
4 x_1+3 x_2-9 x_3=9 \\
2 x_1+3 x_2-5 x_3=\lambda \\
x_1+8 x_2-7 x_3=12
\end{cases}
$$

In [ ]:
import sympy as sp

def gaussian_elimination_parameter(A, b, param_symbol='λ'):
    lam = sp.symbols(param_symbol)
    A = sp.Matrix(A).as_mutable()
    b = sp.Matrix(b).as_mutable()

    # Augmented matrix
    aug = A.row_join(b)
    rows, cols = aug.shape
    n_vars = cols - 1

    print("Initial augmented matrix [A|b]:")
    sp.pprint(aug)
    print("\nStep-by-step elimination:")

    # Gaussian elimination
    for i in range(min(rows, n_vars)):
        if aug[i,i] == 0:
            for k in range(i+1, rows):
                if aug[k,i] != 0:
                    aug.row_swap(i, k)
                    break
        pivot = aug[i,i]
        for j in range(i+1, rows):
            factor = aug[j,i] / pivot
            aug.row_op(j, lambda v, k: v - factor*aug[i,k])

    print("\nRow echelon form [A|b]:")
    sp.pprint(aug)

    # Solve for lambda from last row that has a pivot in a column
    # Look for the first row from bottom that has zeros in all variable columns
    lam_eq = None
    for i in reversed(range(rows)):
        if all(aug[i,j] == 0 for j in range(n_vars)):
            # row has 0 coefficients -> check consistency
            lam_eq = sp.Eq(aug[i,-1], 0)
            break

    if lam_eq is not None:
        lambda_val = sp.solve(lam_eq, lam)
        print(f"\nParameter value(s) that make the system consistent: {lambda_val}")
    else:
        lambda_val = [None]
        print("\nSystem is consistent for all values of the parameter.")

    # Solve for x using back-substitution
    x_solutions = []
    for val in lambda_val:
        if val is not None:
            aug_sub = aug.subs(lam, val)
        else:
            aug_sub = aug
        x = [0]*n_vars
        for i in reversed(range(n_vars)):
            if aug_sub[i,i] == 0:
                continue
            row = aug_sub.row(i)
            s = row[-1] - sum(row[j]*x[j] for j in range(i+1, n_vars))
            x[i] = s / row[i]
        print(f"\nSolution for λ = {val}:")
        for idx, val_x in enumerate(x):
            print(f"x{idx+1} = {sp.simplify(val_x)}")
        x_solutions.append(x)

    return lambda_val, x_solutions

# System
λ = sp.symbols('λ')
A = [
    [2, 5, -8],
    [4, 3, -9],
    [2, 3, -5],
    [1, 8, -7]
]
b = [8, 9, λ, 12]

gaussian_elimination_parameter(A, b, param_symbol='λ')

Initial augmented matrix [A|b]:
⎡2  5  -8  8 ⎤
⎢            ⎥
⎢4  3  -9  9 ⎥
⎢            ⎥
⎢2  3  -5  λ ⎥
⎢            ⎥
⎣1  8  -7  12⎦

Step-by-step elimination:

Row echelon form [A|b]:
⎡2  5   -8     8    ⎤
⎢                   ⎥
⎢0  -7  7      -7   ⎥
⎢                   ⎥
⎢0  0   1    λ - 6  ⎥
⎢                   ⎥
⎢           35   5⋅λ⎥
⎢0  0   0   ── - ───⎥
⎣           2     2 ⎦

Parameter value(s) that make the system consistent: [7]

Solution for λ = 7:
x1 = 3
x2 = 2
x3 = 1


([7], [[3, 2, 1]])

#Задача 2.
Решете матричното уравнение $X A=B$, където

$$
A=\left(\begin{array}{ccc}
1 & 1 & -1 \\
2 & 1 & 0 \\
1 & -1 & 1
\end{array}\right),
$$

$$
B=\left(\begin{array}{ccc}
1 & -1 & 3 \\
4 & 3 & 2 \\
1 & -2 & 5
\end{array}\right)
$$

In [ ]:
import sympy as sp

A = sp.Matrix([
    [1, 1, -1],
    [2, 1, 0],
    [1, -1, 1]
])

B = sp.Matrix([
    [1, -1, 3],
    [4, 3, 2],
    [1, -2, 5]
])

# Solve X = B * A^-1
if A.det() == 0:
    print("Matrix A is singular, cannot compute its inverse.")
else:
    X = B * A.inv()
    print("Solving X = B * A^-1")
    print("\nX = ")
    sp.pprint(X)

    # Verification
    print("\nCheck if X * A = B")
    check = X * A
    sp.pprint(check)

    if sp.simplify(check - B) == sp.zeros(*B.shape):
        print("\nVerification successful: X * A equals B")
    else:
        print("\nVerification failed: X * A does not equal B")

Solving X = B * A^-1

X = 
⎡-3  2  0 ⎤
⎢         ⎥
⎢-4  5  -2⎥
⎢         ⎥
⎣-5  3  0 ⎦

Check if X * A = B
⎡1  -1  3⎤
⎢        ⎥
⎢4  3   2⎥
⎢        ⎥
⎣1  -2  5⎦

Verification successful: X * A equals B


#Задача 3.
Използвайте LU-разлагането на матрицата
$$
A=\left(\begin{array}{cccc}
1 & -1 & 1 & -1 \\
-1 & 3 & -3 & 3 \\
2 & -4 & 7 & -7 \\
-3 & 7 & -10 & 14
\end{array}\right)
$$

и решете системата $A \vec{x}=\vec{b}, \vec{b}=\left(\begin{array}{c}0 \\ 2 \\ -2 \\ 8\end{array}\right)$

In [ ]:
import sympy as sp

# Define matrix A and vector b
A = sp.Matrix([
    [1, -1,  1, -1],
    [-1, 3, -3, 3],
    [2, -4,  7, -7],
    [-3, 7, -10, 14]
])

b = sp.Matrix([0, 2, -2, 8])

# LU decomposition with permutation matrix
L, U, perm = A.LUdecomposition()
P = sp.eye(A.rows).permuteFwd(perm)

# Solve L*y = P*b
Pb = P * b
y = L.solve(Pb)

# Solve U*x = y
x = U.solve(y)

# Display results
print("L =")
sp.pprint(L)

print("\nU =")
sp.pprint(U)

print("\ny =")
sp.pprint(y)

print("\nx =")
sp.pprint(x)

L =
⎡1   0   0   0⎤
⎢             ⎥
⎢-1  1   0   0⎥
⎢             ⎥
⎢2   -1  1   0⎥
⎢             ⎥
⎣-3  2   -1  1⎦

U =
⎡1  -1  1   -1⎤
⎢             ⎥
⎢0  2   -2  2 ⎥
⎢             ⎥
⎢0  0   3   -3⎥
⎢             ⎥
⎣0  0   0   4 ⎦

y =
⎡0⎤
⎢ ⎥
⎢2⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣4⎦

x =
⎡1⎤
⎢ ⎥
⎢1⎥
⎢ ⎥
⎢1⎥
⎢ ⎥
⎣1⎦


#Задача 4.
Намерете собствените стойности и собствените вектори на матрицата

$$A=\left(\begin{array}{ccc}
3 & -2 & -2 \\
-3 & -2 & -6 \\
3 & 6 & 10
\end{array}\right)$$

Проверете, че е диагонализируема, т.е. $\exists$ матрица $S$ такава, че $S^{-1} A S=\operatorname{diag}\left(\lambda_1, \lambda_2, \lambda_3\right), \lambda_i, i=1,2,3$ - собствените стойности.

In [ ]:
import sympy as sp

# Define matrix A
A = sp.Matrix([
    [3, -2, -2],
    [-3, -2, -6],
    [3, 6, 10]
])

# Compute eigenvalues and eigenvectors
eigs = A.eigenvects()
print("Eigenvalues and eigenvectors:")
for val, mult, vecs in eigs:
    print("\nEigenvalue:", sp.simplify(val))
    print("Multiplicity:", mult)
    for v in vecs:
        sp.pprint(v)

# Diagonalization (if possible)
try:
    S, D = A.diagonalize()  # S * D * S.inv() == A
    print("\nMatrix is diagonalizable.\n")

    print("S (modal matrix):")
    sp.pprint(S)

    print("\nD (diagonal matrix of eigenvalues):")
    sp.pprint(D)

    # Verification
    print("\nCheck: S^-1 * A * S = D")
    sp.pprint(sp.simplify(S.inv() * A * S))
except Exception as e:
    print("\nMatrix is not diagonalizable (or symbolic issue):")
    print(e)

Eigenvalues and eigenvectors:

Eigenvalue: 3
Multiplicity: 1
⎡-1/3⎤
⎢    ⎥
⎢ -1 ⎥
⎢    ⎥
⎣ 1  ⎦

Eigenvalue: 4
Multiplicity: 2
⎡-2⎤
⎢  ⎥
⎢1 ⎥
⎢  ⎥
⎣0 ⎦
⎡-2⎤
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎣1 ⎦

Matrix is diagonalizable.

S (modal matrix):
⎡-1  -2  -2⎤
⎢          ⎥
⎢-3  1   0 ⎥
⎢          ⎥
⎣3   0   1 ⎦

D (diagonal matrix of eigenvalues):
⎡3  0  0⎤
⎢       ⎥
⎢0  4  0⎥
⎢       ⎥
⎣0  0  4⎦

Check: S^-1 * A * S = D
⎡3  0  0⎤
⎢       ⎥
⎢0  4  0⎥
⎢       ⎥
⎣0  0  4⎦
